In [36]:
import pandas as pd

data = pd.read_csv("../data/homework_dataset.csv")
print("✅ Original shape:", data.shape)
data.head()

✅ Original shape: (994, 14)


,homework_id,subject,topic,difficulty_1to5,description_sv,due_date,max_points,teacher_name,created_at,steps_count,steps_completeness,reasoning_quality,method_appropriateness,explanation_clarity
0,1,Matematik,Algebra,4,Beräkna värdet av uttrycket 3x² + 2x − 5 när x...,2025-10-12,6,Mats Svensson,2025-09-29,12,0.978,0.921,0.797,0.992
1,2,Matematik,Algebra,2,Beräkna värdet av uttrycket 3x² + 2x − 5 när x...,2025-10-09,5,Maria Holm,2025-09-22,3,0.735,0.606,0.537,0.686
2,3,Matematik,Algebra,2,Beräkna värdet av uttrycket 3x² + 2x − 5 när x...,2025-11-22,7,Karin Bergström,2025-11-04,5,0.634,0.589,0.811,0.627
3,4,Matematik,Algebra,2,Lös ekvationen 4(x − 2) = 8 och förklara varje...,2025-11-23,8,Peter Lindgren,2025-11-11,5,0.895,0.605,0.991,0.838
4,5,Matematik,Algebra,1,Beräkna värdet av uttrycket 3x² + 2x − 5 när x...,2025-09-21,13,Peter Lindgren,2025-09-13,3,0.852,0.732,0.595,0.896


In [37]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Simulate contextual metadata based on difficulty
data["steps_count"] = data["difficulty_1to5"].apply(lambda x: np.random.randint(3, 6) if x <= 2 else np.random.randint(6, 15))
data["steps_completeness"] = data["difficulty_1to5"].apply(lambda x: round(np.random.uniform(0.6, 0.9) if x < 4 else np.random.uniform(0.7, 1.0), 3))
data["reasoning_quality"] = data["difficulty_1to5"].apply(lambda x: round(np.random.uniform(0.5, 0.8) if x < 3 else np.random.uniform(0.7, 1.0), 3))
data["method_appropriateness"] = data["difficulty_1to5"].apply(lambda x: round(np.random.uniform(0.5, 1.0), 3))
data["explanation_clarity"] = data["difficulty_1to5"].apply(lambda x: round(np.random.uniform(0.6, 1.0), 3))


In [38]:
print("✅ New shape:", data.shape)
print(data.head(10))

✅ New shape: (994, 14)
   homework_id    subject    topic  difficulty_1to5  \
0            1  Matematik  Algebra                4   
1            2  Matematik  Algebra                2   
2            3  Matematik  Algebra                2   
3            4  Matematik  Algebra                2   
4            5  Matematik  Algebra                1   
5            6  Matematik  Algebra                2   
6            7  Matematik  Algebra                4   
7            8  Matematik  Algebra                3   
8            9  Matematik  Algebra                4   
9           10  Matematik  Algebra                5   

                                      description_sv    due_date  max_points  \
0  Beräkna värdet av uttrycket 3x² + 2x − 5 när x...  2025-10-12           6   
1  Beräkna värdet av uttrycket 3x² + 2x − 5 när x...  2025-10-09           5   
2  Beräkna värdet av uttrycket 3x² + 2x − 5 när x...  2025-11-22           7   
3  Lös ekvationen 4(x − 2) = 8 och förklara varje..

In [39]:
data.to_csv("../data/homework_dataset.csv", index=False)
print("💾 Updated dataset saved successfully!")

💾 Updated dataset saved successfully!


In [44]:
import joblib
import numpy as np

# --- Load trained models ---
vectorizer = joblib.load("../training_model/tfidf_vectorizer.pkl")
model_topic = joblib.load("../training_model/topic_model.pkl")
model_diff = joblib.load("../training_model/difficulty_model.pkl")

# --- Define safe alignment helper ---
def safe_align_features(X, expected_features):
    """Ensure the input feature matrix matches model feature size."""
    current_features = X.shape[1]
    if current_features < expected_features:
        diff = expected_features - current_features
        X = np.hstack([X, np.zeros((X.shape[0], diff))])
    elif current_features > expected_features:
        X = X[:, :expected_features]
    return X

# --- New example ---
example_text = ["Lös ekvationssystemet: 3x − 2y = 7 och x + y = 5."]
example_max_points = 10  # numeric feature

# --- TF-IDF transform ---
X_text = vectorizer.transform(example_text)
X_combined = np.hstack([X_text.toarray(), np.array([[example_max_points]])])

print("✅ Input shape:", X_combined.shape)
print("✅ Model expects:", model_diff.n_features_in_)

# --- Align feature shape ---
X_combined = safe_align_features(X_combined, model_diff.n_features_in_)

# --- Predict topic & difficulty ---
topic_pred = model_topic.predict(X_text)[0]
difficulty_pred = model_diff.predict(X_combined)[0]

print("\n🔹 Predicted Topic:", topic_pred)
print("🔹 Predicted Difficulty:", difficulty_pred)

✅ Input shape: (1, 112)
✅ Model expects: 117

🔹 Predicted Topic: Ekvationer
🔹 Predicted Difficulty: 2


In [ ]:
import numpy as np
import joblib

# --- Load trained models and vectorizer once ---
vectorizer = joblib.load("../training_model/tfidf_vectorizer.pkl")
model_topic = joblib.load("../training_model/topic_model.pkl")
model_diff = joblib.load("../training_model/difficulty_model.pkl")

def predict_homework_metadata(submission_text: str, max_points: float = 10.0):
    """
    Predict topic and difficulty safely.
    Handles TF-IDF + numeric feature alignment automatically.
    """
    # Step 1: Transform text
    X_text = vectorizer.transform([submission_text])

    # Step 2: Combine with numeric column (max_points)
    X_combined = np.hstack([X_text.toarray(), np.array([[max_points]])])

    # Step 3: Safety check for feature mismatch
    expected_features = model_diff.n_features_in_
    current_features = X_combined.shape[1]

    if current_features != expected_features:
        print(f"⚠️ Feature mismatch: model expects {expected_features}, but got {current_features}")
        print("   Possible cause: vectorizer not from the same training session.")
        print("   Prediction aborted for safety.")
        return None, None

    # Step 4: Predict safely
    topic_pred = model_topic.predict(X_text)[0]
    diff_pred = model_diff.predict(X_combined)[0]

    return topic_pred, diff_pred



In [42]:
X_combined = safe_align_features(X_combined, model_diff.n_features_in_)
difficulty_pred = model_diff.predict(X_combined)[0]

In [43]:
test_examples = [
    "Lös ekvationssystemet: 3x − 2y = 7 och x + y = 5.",
    "Beräkna volymen av en kon med radie 6 cm och höjd 12 cm.",
    "Bestäm minimipunkten för funktionen f(x) = 2x² − 4x + 1.",
    "En bil minskar hastigheten från 90 km/h till 36 km/h på 10 sekunder. Beräkna accelerationen.",
    "En triangel har sidorna 7 cm, 8 cm och 9 cm. Beräkna arean med Herons formel.",
    "En rektangulär låda har volymen 480 cm³. Längden är dubbelt så stor som bredden och höjden är 5 cm. Bestäm måtten.",
    "Beräkna median, medelvärde och spridning för 3, 7, 7, 9, 10, 14, 20.",
    "En investerare placerar 10 000 kr till 4 % ränta per år i 5 år. Hur mycket pengar finns då?",
    "Lös ekvationen 2(x + 3) − (x − 4) = 3x − 5.",
    "Ett företag ökar sin försäljning med 15 % per år i 3 år. Hur mycket har försäljningen ökat totalt?"
]

example_points = [10, 10, 10, 12, 12, 10, 8, 9, 10, 9]

for text, pts in zip(test_examples, example_points):
    X_text = vectorizer.transform([text])
    X_combined = np.hstack([X_text.toarray(), np.array([[pts]])])
    X_combined = safe_align_features(X_combined, model_diff.n_features_in_)

    topic = model_topic.predict(X_text)[0]
    diff = model_diff.predict(X_combined)[0]

    print(f"\n🧩 Text: {text[:40]}...")
    print(f"🔹 Topic: {topic}")
    print(f"🔹 Difficulty: {diff}")


🧩 Text: Lös ekvationssystemet: 3x − 2y = 7 och x...
🔹 Topic: Ekvationer
🔹 Difficulty: 2

🧩 Text: Beräkna volymen av en kon med radie 6 cm...
🔹 Topic: Geometri
🔹 Difficulty: 2

🧩 Text: Bestäm minimipunkten för funktionen f(x)...
🔹 Topic: Funktioner
🔹 Difficulty: 2

🧩 Text: En bil minskar hastigheten från 90 km/h ...
🔹 Topic: Problemlösning
🔹 Difficulty: 1

🧩 Text: En triangel har sidorna 7 cm, 8 cm och 9...
🔹 Topic: Geometri
🔹 Difficulty: 2

🧩 Text: En rektangulär låda har volymen 480 cm³....
🔹 Topic: Geometri
🔹 Difficulty: 2

🧩 Text: Beräkna median, medelvärde och spridning...
🔹 Topic: Statistik & Sannolikhet
🔹 Difficulty: 2

🧩 Text: En investerare placerar 10 000 kr till 4...
🔹 Topic: Problemlösning
🔹 Difficulty: 2

🧩 Text: Lös ekvationen 2(x + 3) − (x − 4) = 3x −...
🔹 Topic: Algebra
🔹 Difficulty: 2

🧩 Text: Ett företag ökar sin försäljning med 15 ...
🔹 Topic: Problemlösning
🔹 Difficulty: 2
